In [15]:
import numpy as np
import os
import pandas as pd
import torch
from gmf import GMF
from data import *

In [16]:
from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Precision, Recall
from ignite.contrib.handlers import ProgressBar

In [24]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


### Data

In [17]:
path = './data/'

In [18]:
interactions = pd.read_csv(path + 't1_train.csv')
human_feats = np.load(path + 'human_feats.npy')
virus_feats = np.load(path + 'virus_feats.npy')

In [19]:
virus_idxs = sorted(interactions['node1'].unique())
human_idxs = sorted(interactions['node2'].unique())
len(virus_idxs), len(human_idxs)

(172, 5225)

In [20]:
vtoi = {v : i for i, v in enumerate(virus_idxs)}
itov = {i : v for i, v in enumerate(virus_idxs)}
htoi = {h : i for i, h in enumerate(human_idxs)}
itoh = {i : h for i, h in enumerate(human_idxs)}

In [21]:
gen = SingleTaskGenerator(interactions, human_feats, virus_feats, .10)

In [8]:
train_loader = gen.create_train_loader(3)
val_loader = gen.create_val_loader(3)
test_loader = gen.create_test_loader(3)

In [9]:
next(iter(train_loader))

[tensor([[ 112, 2907],
         [  44,   40],
         [  59, 3994]]),
 tensor([[0.9500, 0.6000, 0.9000,  ..., 0.0000, 0.0000, 1.0000],
         [0.4000, 0.4000, 0.6667,  ..., 0.0000, 0.0000, 1.0000],
         [0.2857, 0.8286, 0.6286,  ..., 0.0000, 0.0000, 0.0000]],
        dtype=torch.float64),
 tensor([[0.9474, 0.7895, 0.3158,  ..., 0.0000, 0.0000, 1.0000],
         [0.6667, 0.6667, 0.6000,  ..., 0.0000, 0.0000, 1.0000],
         [0.6667, 0.7619, 0.7619,  ..., 0.0000, 0.0000, 1.0000]],
        dtype=torch.float64),
 tensor([0, 0, 0])]

### Model

In [22]:
n_virus, n_human = len(virus_idxs), len(human_idxs)

In [25]:
config = {
    'num_virus': n_virus,
    'num_human': n_human,
    'latent_dim': 50,
    'sparse': False
}
model = GMF(config)
model.to(device)
optimizer = 

GMF(
  (virus): Embedding(172, 50)
  (human): Embedding(5225, 50)
)

In [29]:
optimizer = torch.optim.SGD(model.parameters(), 
                            lr = 1e-3,  
                            momentum=0.9, 
                            weight_decay=1e-5)

### Trainer

In [ ]:
debug_loader = 

In [32]:
def train_batch(engine, batch):
    model.train()
    optimizer.zero_grad()
    
    x_pairs, human_feats, virus_feats, ys = batch

In [31]:
trainer = Engine(train_batch)
trainer.run(train_loader)

[tensor([[  40, 4283],
        [   7, 1185],
        [  16, 3271]]), tensor([[0.2857, 0.2857, 0.3571,  ..., 0.0000, 0.0000, 0.0000],
        [0.6250, 0.5000, 0.1250,  ..., 0.0000, 0.0000, 1.0000],
        [0.3684, 0.5789, 0.1579,  ..., 0.0000, 0.0000, 1.0000]],
       dtype=torch.float64), tensor([[0.1176, 0.3529, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.7500, 0.9375, 0.6875,  ..., 0.0000, 0.0000, 1.0000],
        [0.3125, 0.8125, 0.3750,  ..., 0.0000, 0.0000, 1.0000]],
       dtype=torch.float64), tensor([0, 0, 0])]
[tensor([[  24, 3321],
        [ 123, 4585],
        [  85,  787]]), tensor([[0.7059, 0.7647, 0.7647,  ..., 0.0000, 0.0000, 1.0000],
        [0.8000, 0.7333, 0.9333,  ..., 0.0000, 0.0000, 0.0000],
        [0.1000, 0.9500, 0.1500,  ..., 0.0000, 0.0000, 1.0000]],
       dtype=torch.float64), tensor([[0.9688, 0.6875, 0.4375,  ..., 0.0000, 0.0000, 1.0000],
        [0.9910, 0.8108, 0.5586,  ..., 0.0000, 0.0000, 1.0000],
        [0.9600, 0.6400, 0.9200,  ..., 0.0000, 0.